https://data.census.gov/mdat/#/search?ds=ACSPUMS1Y2018

https://www.prb.org/workingaroundtheclock/

https://www.census.gov/quickfacts/fact/table/madisoncitywisconsin/LND110210

https://www2.census.gov/programs-surveys/acs/data/pums/2018/1-Year/PUMS_file_naming_convention.pdf



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
DemoGraphicDataPATH = "/Users/megantabbutt/BRT_data/DemographicData/csv_pwi/psam_p55_5.csv"
TimeTableConversionPATH = "/Users/megantabbutt/BRT_data/DemographicData/TimeConversiontable.csv"

In [ ]:
timeConversionTable = pd.read_csv(TimeTableConversionPATH, header=None, index_col=1)
DemoGraphicData = pd.read_csv(DemoGraphicDataPATH)

In [ ]:
DemoGraphicData

In [ ]:
timeConvertDict = timeConversionTable.to_dict()[0]

In [ ]:
whiteBoolean = list(DemoGraphicData['RACWHT'])
print("Ratio white: ", np.average(whiteBoolean))

blackBoolean = list(DemoGraphicData['RACBLK'])
print("Ratio black: ", np.average(blackBoolean))

hispanicBoolean = [1 if i > 1 else 0 for i in list(DemoGraphicData['HISP'])]
print("Ratio hispanic: ", np.average(hispanicBoolean))

asianBoolean = (DemoGraphicData['RACASN'])
print("Ratio asian: ", np.average(asianBoolean))

In [ ]:
whiteBoolean_TF = []
for i in whiteBoolean:
    if i==1:
        whiteBoolean_TF.append(True)
    else:
        whiteBoolean_TF.append(False)
print(np.average(whiteBoolean_TF))
whiteData = DemoGraphicData[whiteBoolean_TF]

In [ ]:
blackBoolean_TF = []
for i in blackBoolean:
    if i==1:
        blackBoolean_TF.append(True)
    else:
        blackBoolean_TF.append(False)
print(np.average(blackBoolean_TF))
blackData = DemoGraphicData[blackBoolean_TF]

In [ ]:
hispanicBoolean_TF = []
for i in hispanicBoolean:
    if i==1:
        hispanicBoolean_TF.append(True)
    else:
        hispanicBoolean_TF.append(False)
print(np.average(hispanicBoolean_TF))
hispanicData = DemoGraphicData[hispanicBoolean_TF]

In [ ]:
asianBoolean_TF = []
for i in asianBoolean:
    if i==1:
        asianBoolean_TF.append(True)
    else:
        asianBoolean_TF.append(False)
print(np.average(asianBoolean_TF))
asianData = DemoGraphicData[asianBoolean_TF]

In [ ]:
hours = [i for i in range(24)]

def getDictHours(data):
    DataHours = dict()
    for i in hours:
        DataHours[i] = 0

    for i in data:
        if i >=1.0 and i <= 150.0:
            hourTime = timeConvertDict[int(i)]
            DataHours[hourTime] += 1
    return DataHours
            
whiteDataHours = getDictHours(whiteData['JWDP'])
blackDataHours = getDictHours(blackData['JWDP'])
hispanicDataHours = getDictHours(hispanicData['JWDP'])
asianDataHours = getDictHours(asianData['JWDP'])

In [ ]:
# Get the wieghts:

DemographicsWeightsHours = {'White':[], 'Black':[], 'Hispanic':[], 'Asian':[]}
ErrorsUpper = {'White':[], 'Black':[], 'Hispanic':[], 'Asian':[]}
ErrorsLower = {'White':[], 'Black':[], 'Hispanic':[], 'Asian':[]}

totalPeople = len(DemoGraphicData)
expectedWhitesRatio = np.average(whiteBoolean_TF)
expectedBlacksRatio = np.average(blackBoolean_TF)
expectedHispanicRatio = np.average(hispanicBoolean_TF)
expectedAsianRatio = np.average(asianBoolean_TF)

for hour in hours:
    wNum = whiteDataHours[hour]
    bNum = blackDataHours[hour]
    hNum = hispanicDataHours[hour]
    aNum = asianDataHours[hour]
    totalHourWorkers = wNum + bNum + hNum + aNum
    
    wWeight = wNum / (totalHourWorkers * expectedWhitesRatio)
    bWeight = bNum / (totalHourWorkers * expectedBlacksRatio)
    hWeight = hNum / (totalHourWorkers * expectedHispanicRatio)
    aWeight = aNum / (totalHourWorkers * expectedAsianRatio)
    
    DemographicsWeightsHours['White'].append(wWeight)
    DemographicsWeightsHours['Black'].append(bWeight)
    DemographicsWeightsHours['Hispanic'].append(hWeight)
    DemographicsWeightsHours['Asian'].append(aWeight)
    
    ErrorsUpper['White'].append(np.sqrt(wNum)/ ((totalHourWorkers + np.sqrt(wNum)) * expectedWhitesRatio))
    ErrorsUpper['Black'].append(np.sqrt(bNum)/ ((totalHourWorkers + np.sqrt(bNum)) * expectedBlacksRatio))
    ErrorsUpper['Hispanic'].append(np.sqrt(hNum)/ ((totalHourWorkers + np.sqrt(hNum)) * expectedHispanicRatio))
    ErrorsUpper['Asian'].append(np.sqrt(aNum)/ ((totalHourWorkers + np.sqrt(aNum)) * expectedAsianRatio))
    
    ErrorsLower['White'].append(np.sqrt(wNum)/ ((totalHourWorkers - np.sqrt(wNum)) * expectedWhitesRatio))
    ErrorsLower['Black'].append(np.sqrt(bNum)/ ((totalHourWorkers - np.sqrt(bNum)) * expectedBlacksRatio))
    ErrorsLower['Hispanic'].append(np.sqrt(hNum)/ ((totalHourWorkers - np.sqrt(hNum)) * expectedHispanicRatio))
    ErrorsLower['Asian'].append(np.sqrt(aNum)/ ((totalHourWorkers - np.sqrt(aNum)) * expectedAsianRatio))

#DemographicsWeightsHours

In [ ]:
hoursWhite = [i for i in hours]
hoursBlack = [i-.08 for i in hours]
hoursHispanic = [i+.08 for i in hours]

In [ ]:
totalWorkersHours = []
for hour in hours:
    wNum = whiteDataHours[hour]
    bNum = blackDataHours[hour]
    hNum = hispanicDataHours[hour]
    aNum = asianDataHours[hour]
    totalWorkersHours.append(wNum + bNum + hNum + aNum)
totalWorkersHours = [i*(2/max(totalWorkersHours)) for i in totalWorkersHours]

In [ ]:
fig, ax1 = plt.subplots(figsize=(20, 8))

#ax = plt.subplots(figsize=(20, 8))[1]


ax1.bar(hours, totalWorkersHours, color="#e7e3e0", alpha=.6, zorder=1)

ax2 = ax1.twinx()
ax2.errorbar(hours, DemographicsWeightsHours['White'], yerr=[ErrorsUpper['White'], ErrorsLower['White']], 
             fmt='o', label='White', markersize=8, color="#89d348", zorder=3)

ax2.errorbar(hoursBlack, DemographicsWeightsHours['Black'], yerr=[ErrorsUpper['Black'], ErrorsLower['Black']], 
             fmt='o', label='Black', markersize=8, color='#3f788f', zorder=2)

ax2.errorbar(hoursHispanic, DemographicsWeightsHours['Hispanic'], 
             yerr=[ErrorsUpper['Hispanic'], ErrorsLower['Hispanic']], fmt='o', label='Hispanic', 
             markersize=8, color='#410056', zorder=1)

#plt.plot(hours, DemographicsWeightsHours['Asian'], color='purple', label='Asian')
#plt.scatter(hours, DemographicsWeightsHours['Asian'], color='purple')

ax2.set_yticks([])
ax2.set_ylabel("Counts of People (arb)", fontsize=16)


hourLabels = ["12am", " ", "2am", " ", "4am", " ", "6am", " ", "8am", " ", "10am", " ",
             "12pm", " ", "2pm", " ", "4pm", " ", "6pm", " ", "8pm", " ", "10pm", " "]
plt.xticks(hoursWhite, labels=hourLabels)
ax1.grid(axis='x', linestyle=':', alpha=.6, zorder=0)
plt.title("How likely an ethnic group is to leave for work at a given time", fontsize=18)
plt.xlabel("Hour that group leaves to go to work", fontsize=16)
ax1.set_ylabel("Weight Factor", fontsize=16)
plt.ylim(0, 2.2)
ax2.legend(fontsize=14, bbox_to_anchor=(0.12, .98))
plt.savefig("PaperFigures/WeightFactorRaces", bbox_inches='tight', format='svg')
plt.savefig("PaperFigures/WeightFactorRaces", bbox_inches='tight')
plt.show()